In [1]:
import pandas as pd
import numpy as np
import csv
import joblib

from tensorflow import keras
from matplotlib import pyplot as plt
from itertools import cycle

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV        
from sklearn.ensemble import RandomForestClassifier

2023-10-04 17:44:37.218503: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
data = pd.read_csv("data.csv")
data.head()

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


**Translate the labels**

In [4]:

normal_file = "data.csv"
updated_file = "re_label_name.csv"

translation_dict = {'rice' : 'ข้าว', 'maize' : 'ข้าวโพด', 'chickpea' : 'ถั่วลูกไก่', 'kidneybeans' : 'ถั่วแดงหลวง ', 
                   'pigeonpeas' : 'ถั่วแระ', 'mothbeans' : 'มัตกี', 'mungbean' : 'ถั่วเขียว', 'blackgram' : 'ถั่วดํา', 
                   'lentil' : 'ถั่วลิสง', 'pomegranate' : 'ทับทิม', 'banana' : 'กล้วย', 'mango' : 'มะม่วง', 'grapes' : 'องุ่น', 
                   'watermelon' : 'แตงโม', 'muskmelon' : 'แตงไทย', 'apple' : 'แอปเปิ้ล', 'orange' : 'ส้ม', 'papaya' : 'มะละกอ', 
                   'coconut' : 'มะพร้าว', 'cotton' : 'ฝ้าย', 'jute' : 'ปอแก้ว', 'coffee' : 'กาแฟ'}

data["translation_label"] = data["label"].map(translation_dict)
print(data["translation_label"].unique())



['ข้าว' 'ข้าวโพด' 'ถั่วลูกไก่' 'ถั่วแดงหลวง ' 'ถั่วแระ' 'มัตกี'
 'ถั่วเขียว' 'ถั่วดํา' 'ถั่วลิสง' 'ทับทิม' 'กล้วย' 'มะม่วง' 'องุ่น'
 'แตงโม' 'แตงไทย' 'แอปเปิ้ล' 'ส้ม' 'มะละกอ' 'มะพร้าว' 'ฝ้าย' 'ปอแก้ว'
 'กาแฟ']


In [5]:
def is_fertile (ph, temp, humidity) :
    ph_range = (6.0, 7.0)
    temp_range = (20.0, 30.0)
    humidity_range = (50.0, 80.0)
    
    ph_in_range = ph_range[0] <= ph <=ph_range[1]
    temp_in_range = temp_range[0] <= temp <= temp_range[1]
    humidity_in_range = humidity_range[0] <= humidity <= humidity_range[1]
    
    if ph_in_range and temp_in_range and humidity_in_range :
        return "Fertile soil"
    else :
        return "Non-fertile soil"

In [6]:
data = data.drop(columns=['N', 'P', 'K', "rainfall", 'label'])
data.head()

,temperature,humidity,ph,translation_label
0,20.879744,82.002744,6.502985,ข้าว
1,21.770462,80.319644,7.038096,ข้าว
2,23.004459,82.320763,7.840207,ข้าว
3,26.491096,80.158363,6.980401,ข้าว
4,20.130175,81.604873,7.628473,ข้าว


In [7]:
data.size

8800

In [8]:
data.shape

(2200, 4)

In [9]:
data["translation_label"].unique()

array(['ข้าว', 'ข้าวโพด', 'ถั่วลูกไก่', 'ถั่วแดงหลวง ', 'ถั่วแระ',
       'มัตกี', 'ถั่วเขียว', 'ถั่วดํา', 'ถั่วลิสง', 'ทับทิม', 'กล้วย',
       'มะม่วง', 'องุ่น', 'แตงโม', 'แตงไทย', 'แอปเปิ้ล', 'ส้ม', 'มะละกอ',
       'มะพร้าว', 'ฝ้าย', 'ปอแก้ว', 'กาแฟ'], dtype=object)

In [10]:
features = data.drop(columns=["translation_label"])
target = data["translation_label"]

**Train model**

In [11]:
X_train, x_val, Y_train, y_val = train_test_split(features, target, test_size=0.2, random_state=42)

In [12]:
param_distribution_rand_forests = {'n_estimators': range(5, 50),  'max_depth': range(3, 30)}
model = GridSearchCV(RandomForestClassifier(), param_distribution_rand_forests, scoring='balanced_accuracy', n_jobs=-1)
model.fit(X_train, Y_train)

GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': range(3, 30),
                         'n_estimators': range(5, 50)},
             scoring='balanced_accuracy')

In [13]:
results = model.cv_results_

n_estimators = results['param_n_estimators'].data.astype(int)
max_depth = results['param_max_depth'].data.astype(int)
mean_test_score = results['mean_test_score']

mean_test_score = np.array(mean_test_score).reshape(len(param_distribution_rand_forests['max_depth']), -1)

In [14]:
best_params = model.best_params_
best_estimator = model.best_estimator_
best_score = model.best_score_ * 100

print("Best Hyperparameters:", best_params)
print("Best Balanced Accuracy:", best_score, "%")

Best Hyperparameters: {'max_depth': 10, 'n_estimators': 47}
Best Balanced Accuracy: 75.15113318054493 %


In [67]:
data_from_arduino = pd.read_csv("moisture_data.csv")
data_from_arduino.head()

test_data = data_from_arduino[['temperature', 'humidity', 'ph']].values
test_data = np.array(test_data)

test_data = test_data.tolist()

test_data

[[20.130175, 1.0, 6.464925], [26.49, 1.0, 7.517578]]

**Predict**

In [68]:

prediction = model.predict(test_data)
prediction = np.array(prediction)
prediction_str = ' '.join(prediction)

print(prediction_str)

ถั่วลูกไก่ ถั่วลูกไก่


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


**Check if the data is Fertile or not**

In [69]:
def is_fertile_soil(temp, humidity, ph):
    ph_range = (6.0, 7.5)  
    temp_range = (20.0, 30.0)  
    humidity_range = (50.0, 80.0)  

    ph_in_range = ph_range[0] <= ph <= ph_range[1]
    temp_in_range = temp_range[0] <= temp <= temp_range[1]
    humidity_in_range = humidity_range[0] <= humidity <= humidity_range[1]

    if ph_in_range and temp_in_range and humidity_in_range:
        return "สุขภาพดี"
    else:
        return "สุขภาพแย่"

In [74]:
ph_value = test_data[0][2]
temp_value = test_data[0][0]
humidity_value = test_data[0][1]

**Function return to txt file**

In [75]:
res = is_fertile_soil(temp_value, humidity_value, ph_value)

output_dict = {'เราแนะนำให้คุณลองปลูก': prediction_str,
            'ค่า pH ที่ตรวจได้': ph_value,
            'อุณหภูมิ ที่ตรวจได้': temp_value,
            'ความชื่น ที่ตรวจได้': humidity_value,
            'เป็นดินที่': res}

output_dict

{'เราแนะนำให้คุณลองปลูก': 'ถั่วลูกไก่ ถั่วลูกไก่',
 'ค่า pH ที่ตรวจได้': 6.464925,
 'อุณหภูมิ ที่ตรวจได้': 20.130175,
 'ความชื่น ที่ตรวจได้': 1.0,
 'เป็นดินที่': 'สุขภาพแย่'}

In [76]:
import shutil

file_name = "rec_output.txt"
flash_drive_path = "/Volumes/AI"  # Replace with the actual path to your flash drive

# Write data to the text file
with open(file_name, "w", encoding="utf-8") as file:
    for key, value in output_dict.items():
        file.write(f"{key}: {value}")

print(f"Data has been written to {file_name}")

# Copy the text file to the flash drive
try:
    if os.path.exists(file_name) and os.path.exists(flash_drive_path):
        shutil.copy(file_name, os.path.join(flash_drive_path, file_name))
        print(f"'{file_name}' has been copied to the flash drive.")
    else:
        print("File or flash drive path does not exist.")
except Exception as e:
    print(f"An error occurred: {str(e)}")


Data has been written to rec_output.txt
File or flash drive path does not exist.


**Save model to tflite for use**

In [59]:
best_model = model.best_estimator_

In [61]:
joblib.dump(best_model, 'rec_crop_with_nutrition.tflite')

['rec_crop_with_nutrition.tflite']